# Model from the Saranya paper

## Imports

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

import os

#NN
import keras
from keras import layers
from keras.models import Sequential

from sklearn.model_selection import train_test_split
import sklearn.metrics as metrics

from sklearn.preprocessing import OneHotEncoder

# setting seed 
np.random.seed(seed=42)

## Images preview

In [ ]:
i = 3
img_1 = cv2.imread(f"seg_dataset_saranya2021/Green/{str(i)}.jpg")
img_2 = cv2.imread("seg_dataset_saranya2021/Overripen/2.jpg")
print("image 1 shape (lines, columns, channels) = ", img_1.shape) 
print("image 2 shape (lines, columns, channels) = ", img_2.shape) 
print("\nimage type: ", type(img_1))

In [ ]:
rgb_img_1 = cv2.cvtColor(img_1, cv2.COLOR_BGR2RGB) # BGR to RGB 
rgb_img_2 = cv2.cvtColor(img_2, cv2.COLOR_BGR2RGB) # BGR to RGB 
plt.imshow(rgb_img_1)
#plt.axis('off')
plt.show()

plt.imshow(rgb_img_2)
#plt.axis('off')
plt.show()

## Data Pre-processing

### dataset Saranya

In [ ]:
# paths to the image folders of each class
class1_path = "seg_dataset_saranya2021/Green"
class2_path = "seg_dataset_saranya2021/Midripen"
class3_path = "seg_dataset_saranya2021/Overripen"
class4_path = "seg_dataset_saranya2021/Yellowish_Green"

# Define empty lists to store the images and labels
images = []
labels = []

# Loop through each folder and extract the images
for class_path, class_label in [(class1_path, 0), (class2_path, 1), (class3_path, 2), (class4_path, 3)]:
    for image_file in os.listdir(class_path):
        # Load the image and convert to RGB format
        image_path = os.path.join(class_path, image_file)
        image = cv2.imread(image_path)
        image = cv2.resize(image, (64, 64))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
        
        # Add the image and label to the lists
        images.append(image)
        labels.append(class_label)

# Convert the images and labels to NumPy arrays
X = np.array(images)
y = np.array(labels)

In [ ]:
plt.imshow(cv2.cvtColor(X[0], cv2.COLOR_HSV2RGB))
#plt.axis('off')
plt.show()

In [ ]:
#from collections import Counter
#print(sorted(Counter(y).items()))

### Spliting the data

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.4, 
                                                    shuffle= True,
                                                    random_state=42)


### Balancing data with weights

In [ ]:
# takes a vector with labels y, 
# returns the proper class weights 
# to be used in the fit function from keras
# to use in the training of the model
# prints the no. of classes

# returned variable to be used in training keras model

def get_weights(yy):
    y=yy
    # no. of elements per class
    unique_classes, class_counts = np.unique(y, return_counts=True, axis=0)
    print(f"unique_classes-->{unique_classes}")
    # Print the unique classes and their corresponding counts
    for cls, count in zip(unique_classes, class_counts):
        print(f"Class {cls}: {count} instances")

    total_instances = np.sum(class_counts)
    class_weights = total_instances / (class_counts * len(unique_classes))

    # Print the class weights
    for cls, weight in zip(unique_classes, class_weights):
        print(f"Class {cls} weight: {weight}")

    class_weights /= np.sum(class_weights)

    # Print the normalized class weights
    for cls, weight in zip(unique_classes, class_weights):
        print(f"Class {cls} weight (normalized): {weight}")


    w_dict = {}
    for i in range(len(unique_classes)):
        w_dict[i] = class_weights[i]
    
    return w_dict

In [ ]:
w = get_weights(y_train)
w

In [ ]:
s = get_weights(y_test)

## Data Augmentation

Only applying data augmentation to the training data so that the model is not being evaluated on artifiacilly generated data

In [ ]:
a = np.random.randint(20)
a = [0, 0, 0, 0]
np.allclose(a, 0)

In [ ]:
def data_augmentation(X, y, num_augmented_samples = 100):
    augmented_X = []
    augmented_y = []

    # Obtain the number of original samples
    num_samples = X.shape[0]

    # Calculate the number of augmented samples needed for each class
    unique_classes, label_counts = np.unique(y, return_counts=True, axis=0)
    max_label_count = np.max(label_counts)

    num_augmented_per_class = (max_label_count - label_counts) + num_augmented_samples # // num_augmented_samples

    num_augmented_per_class = np.array(num_augmented_per_class, dtype=int)

    while np.any(num_augmented_per_class):

        ran = np.random.randint(num_samples)

        image = X[ran]
        label = y[ran]

        if num_augmented_per_class[label] > 0:

            flip = np.random.choice(['y', 'n'])

            angle = np.random.uniform(-90, 90)
            rotated_image = rotate_image(image, angle)

            tx = np.random.randint(-10, 10)
            ty = np.random.randint(-10, 10)
            translated_image = translate_image(rotated_image, tx, ty)

            brightness_factor = np.random.uniform(0.5, 1.3)
            augmented_image = adjust_brightness(translated_image, brightness_factor)

            if flip == 'y':
                augmented_image = flip_image(augmented_image, flip_code=1)

            #plt.imshow(cv2.cvtColor(augmented_image, cv2.COLOR_HSV2RGB))
            #plt.show()

            augmented_X.append(augmented_image)
            augmented_y.append(label)

            num_augmented_per_class[label] -= 1
        
    augmented_X = np.array(augmented_X)
    augmented_y = np.array(augmented_y)

    return np.append(X, augmented_X, axis=0), np.append(y, augmented_y, axis=0)

def adjust_brightness(image, factor):
    image = image.astype(np.float32)
    image[:, :, 2] *= factor
    image[:, :, 2] = np.clip(image[:, :, 2], 0, 255)
    return image.astype(np.uint8)

def rotate_image(image, angle):
    height, width, _ = image.shape
    center = (width // 2, height // 2)
    rotation_matrix = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated_image = cv2.warpAffine(image, rotation_matrix, (width, height))
    return rotated_image

def flip_image(image, flip_code):
    flipped_image = cv2.flip(image, flip_code)
    return flipped_image

def translate_image(image, tx, ty):
    translation_matrix = np.float32([[1, 0, tx], [0, 1, ty]])
    translated_image = cv2.warpAffine(image, translation_matrix, (image.shape[1], image.shape[0]))
    return translated_image


In [ ]:
X_aug, y_aug = data_augmentation(X_train, y_train, 200)

In [ ]:
get_weights(y_aug)

## Normalizing input data to improve training speed and OHE

In [ ]:
hsv_max = [179, 255, 255]

X_aug_norm = np.divide(X_aug, hsv_max)

from keras.utils import to_categorical

# Convert labels to one-hot encoding
y_aug_hot = to_categorical(y_aug)

## CNN

In [ ]:
model = Sequential()

# layers.Conv2D(# of filters, filter dimension, ...)
# padding = 'valid' means no padding
model.add(layers.Conv2D(32, 3, activation = 'relu', padding="valid", input_shape = X[0].shape))
model.add(layers.Conv2D(16, 5, activation = 'relu', strides=(1,1), padding="valid"))
model.add(layers.Conv2D(16, 5, activation = 'relu', padding="valid"))

model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(16, 3, activation = 'relu', padding="valid"))
model.add(layers.Conv2D(16, 5, activation = 'relu', padding="valid"))

model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Dropout(0.2))

model.add(layers.Flatten())

model.add(layers.Dense(units = 16, activation = 'relu'))
model.add(layers.Dense(units = 4, activation = 'softmax'))

opt_1 = keras.optimizers.Adam(learning_rate=0.005)

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.summary()

callbacks, only using one

In [ ]:
cb_early = keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta=0.001,
    patience=10,
    verbose=1,
    mode="auto",
    baseline=None,
    restore_best_weights=True,
)

cb_plateau = keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.8,
    patience=3,
    verbose=1,
    mode="auto",
    min_delta=0.001,
    cooldown=4,
    min_lr=1e-4,
)

In [ ]:
bf = model.fit(X_aug_norm, y_aug_hot, validation_data=(X_test_norm, y_test_hot),# class_weight = w,
               epochs=50, batch_size=16, callbacks=[cb_early, cb_plateau])

## Results

In [ ]:
X_test_norm = np.divide(X_test, hsv_max)
y_test_hot = to_categorical(y_test)

# best epoch obtained
results_test = model.evaluate(X_test_norm, y_test_hot)
print(results_test)

In [ ]:
# summary of the model applied to the validation set
y_preds = model.predict(X_test_norm)

y_max = np.max(y_preds, axis=1)
y_p = np.zeros(y_preds.shape)

for i in range(y_p.shape[0]):
    y_p[i, (np.where(y_preds[i]==np.max(y_preds[i])))] = 1

#print(y_p)

print(metrics.classification_report(y_test_hot, y_p))

In [ ]:
loss = bf.history['loss']
val_loss = bf.history['val_loss']
epochs = range(1, len(loss) + 1)

plt.plot(epochs, loss, 'r', label = 'Training set')
plt.plot(epochs, val_loss, 'b', label = 'Validation set')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show

In [ ]:
loss = bf.history['accuracy']
val_loss = bf.history['val_accuracy']
epochs = range(1, len(loss) + 1)


plt.plot(epochs, loss, 'r', label = 'Training set')
plt.plot(epochs, val_loss, 'b', label = 'Validation set')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show

In [ ]:
#model.save("n_gen.h5")